In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# uncomment below if you want bigger resolution images
#plt.rcParams['figure.dpi'] = 200

## Image Analysis Basics
In this notebook, we will learn how to do basic image operations such as thresholding, segmentation and smoothing/denoising of images with basic python functionalities. Here are some of the specific learning objectives:

1. Loading and plotting images and datasets with python commands
2. Array operations with images such as thresholding (usually the first step in most imaging analysis workflow)
3. Basic theory behind convolution and use it to create filters. We will use these to:
    * Explore the effects of various kinds of local averaging on an image;
    * Use specialized filters to emphasize specific features in an image.
4. Use statistical tools to do a basic image classification problem (an example of classical Machine Learning)

### Image and data files
The data files you will need for this notebook can be found on the catcourses module page for Day 4.  Please copy the contents of that folder into your working directory i.e. where this notebook is hosted. First let's look at the contents of "bwCat.tif". These files were originally downloaded from http://physicalmodelingwithpython.blogspot.com/p/data-sets.html from the datasets 16 and 17, which also includes the README files.

In [2]:
# read in the image file. This is usually in formats such as tif, jpg, png,.....
photo = plt.imread('bwCat.tif')

Do the following to check the attributes of the variable called "photo"

1.Output it explicitly and see that it is an array of numbers.

2.Find the shape of the array using photo.shape (that corresponds to the number of pixels) 

3.Check the type of the array using photo.dtype 

4.recap how to take a slice of the array, say the top left square of ten elements

In [2]:
# execute plt.imshow(photo) to actually see the cat!
#plt.imshow(photo)

168

Let us switch to gray scale because that's what the simple image analysis tools are applicable to.

In [3]:
#plt.set_cmap('gray') # Use grayscale for black and white image.
#plt.axis('off') # Get rid of axes and tick marks.
#fig = plt.gcf() # Get current figure object.
#fig.set_facecolor('white') # Set border color to b
#plt.imshow(photo)

Create a binary image called "new_cat" by choosing lighter pixels to be ones and darker pixels to be zero.  This is a common strategy to do thresholding on an image, i.e. isolate the darker and lighter parts of an image, which in turn can be a strategy to segmenting of an image, i.e. to separate it into discrete objects, such as background and foreground.

In [4]:
#convert to binary
#new_cat = (photo > photo.mean())
#new_cat

In [ ]:
plt.imshow(new_cat)

## Convolution: Image Smoothing
If you've ever played with photographic software such as GIMP(or a commercial alternative like Photoshop), you've probably used *convolutions* to smooth or sharpen, images but perhaps you didn't know it.
The mathematical definition of convolving two functions (originally comes from probability theory and gives the probability distribution of the sum of two random variables each with its own probability distribution)
$$ (f*g)(n) = \Sigma_l f(l) g(n-l) $$
where the value of the indices, l and n-l, in the summation, range over all possible values where the functions are define.
So for example f is a 1D array of size L, and g is a 1D array of size N, the index of the array (f*g) can range to nmax, such that $nmax-L = N-1$, or in other words, the size of the convolved array is $N+L-1$.
In the image analysis context, we convolve the image array I with a filter array F in the following way:
$$ C_{i,j} = \Sigma_l F_{k,l} I_{i-k, j-l}$$

A visual illustration of this is offered by the following:
<img src="stride1.gif" width="400" height="480">
which shows intuitively what multiplying by F does. It averages every element of I with those of its neighbors with weights specified by F.

Two important identities follow:
1. The trivial transformation, for which F is a 1by1 matrix with
a single entry equal to 1, is identity i.e. it gives back the original image.
2. Suppose that the size of F is $m \times n$, and that of I is $M \times N$, the size of C is $(M + m - 1) \times (N + n - 1)$.

In [5]:
import scipy.ndimage as sim  # SciPy image library has built-in convolution functions

In [17]:
sim.convolve?

In this lab, you will explore several filters and convolutions. As you proceed through the exercises,
you will see a photograph transformed by each operation. To get an idea of what is happening on a
pixel-by-pixel level, you can apply the same convolution to a single dot. (This is the impulse response
of the filter.) This will allow you to see the shape of the filter and better understand what you are
doing to the photograph. Try the impuls-response below.
Use the subplots function to plot the impulse and the response side-by-side.

In [18]:
impulse = np.zeros( (51, 51) )
impulse[25, 25] = 1.0
my_filter = np.ones( (3, 3) ) / 9
response = sim.convolve(impulse, my_filter)

In [ ]:
plt.figure()
plt.subplot(121)
plt.imshow(impulse)
plt.subplot(122)
plt.imshow(response)

The above was an example of a square filter. Now define a larger square filter by 15 by 15. Apply both small and large square filters
to the cat image stored in the variable *photo* using sim.convolve and plot the small and large square filter results side-by-side.

In [ ]:
large_square_filter = np.ones((15,15))/225
small_square_filter = np.ones( (3, 3) ) / 9

photo_large_square = sim.convolve(photo,large_square_filter )
photo_small_square = sim.convolve(photo,small_square_filter )

plt.figure()
plt.subplot(121)
plt.imshow(photo_small_square)
plt.subplot(122)
plt.imshow(photo_large_square)

### gaussian filter

Now we will look at a slightly more complex filter, called a Gaussian filter. L

a.Load the provided file gauss_filter.csv into a NumPy array called gauss.

b. Use plt.imshow to compare the convolutions of a single dot with a Gaussian filter and the
square filter you used before.

In [23]:
# load gauss_filter.csv
gauss= np.loadtxt('gauss_filter.csv', delimiter = ',')

In [5]:
# compare Gaussian and square filters on single dot (impulse defined before)
plt.figure()
plt.subplot(121)
plt.imshow(sim.convolve(impulse,gauss))
plt.subplot(122)
plt.imshow(sim.uniform_filter(impulse,5))

### Emphasizing features

In between \features" (real things of interest to us) and \noise" (random things), experimental
images may contain things that are real, but not of interest to us. We may wish to deemphasize
such things, or we may wish to quantify some visual feature.
As an example, we look at how Zemel and coauthors sought to quantify the extent to which the
cell was polarized, at every point in the cell (https://www.nature.com/articles/nphys1613).
We work with the data set 17stressFibers,
README.txt and stressFibers.csv which should be in your working directory.  The image shows the long, slender stress fibers. We will now construct and apply a filter that
emphasizes long, slender objects that are oriented vertically.

In [ ]:
#np.loadtxt and plot the image
stressFibers = np.loadtxt('stressFibers.csv', delimiter = ',')


Now we make our custom made filter to emphasize elongated slender structures and test it out.

In [ ]:
# convolution.py [get code]
v = np.arange(-25, 26)
X, Y = np.meshgrid(v, v)
gauss_filter = np.exp(-0.5*(X**2/2 + Y**2/45))
laplace_filter = np.array( [[0, -1, 0], [-1, 4, -1], [0, -1, 0]])
combined_filter  = sim.convolve(gauss_filter, laplace_filter)
#%% Create a plus sign '+' to demonstrate effect of filter.
plus = np.zeros((51, 51))
plus[23:28, 25] = 1.0
plus[25, 23:28] = 1.0

plt.figure()
plt.imshow(plus)
plt.gray()

#%%	Apply filter to '+' and display resulting image.
#	Use vmin/vmax to emphasize features within a restricted range of intensity.
cplus = sim.convolve(plus, combined_filter)

plt.figure()
plt.imshow(cplus, vmin=0, vmax=0.5*cplus.max())
plt.gray()

plt.show()

In [8]:
#rescale array values to between 0 and 255
#stressFibers.shape
new = 255*(stressFibers - np.min(stressFibers))/np.max(stressFibers)
plt.imshow(new)

Now use sim.convolve to apply the filter to the fiber image, display your results, and comment.

In [ ]:
cnew = sim.convolve(new, combined_filter)
plt.imshow(cnew, vmin=0, vmax=0.5*cnew.max())

In [9]:
#make horizontal filter

#apply to stressFibers and plot


In [10]:
# make 45 degrees filter


## Image classification using scikitlearn

In [11]:
import sklearn

from IPython.core.display import Image, display
display(Image(filename= 'iris_setosa.jpg'))
print("Iris Setosa\n")

display(Image(filename='iris_versicolor.jpg'))
print("Iris Versicolor\n")

display(Image(filename='iris_virginica.jpg'))
print("Iris Virginica")

In [14]:
from sklearn.datasets import load_iris
iris = load_iris()

print(iris.data.shape)
print(iris.target.shape)

(150, 4)
(150,)


In [ ]:
# Choose any two features so that the data can be represented as a 2D plot
x_index = 0
y_index = 1

# this formatter will label the colorbar with the correct target names
formatter = plt.FuncFormatter(lambda i, *args: iris.target_names[int(i)])

plt.scatter(iris.data[:, x_index], iris.data[:, y_index],
            c=iris.target, cmap=plt.cm.get_cmap('RdYlBu', 3))
plt.colorbar(ticks=[0, 1, 2], format=formatter)
plt.clim(-0.5, 2.5)
plt.xlabel(iris.feature_names[x_index])
plt.ylabel(iris.feature_names[y_index])

### Quick Exercise:

In the cells below,
**Change** `x_index` **and** `y_index` in the above script
and find a combination of two parameters
which maximally separate the three classes. Try two cases.

This exercise is a preview of **dimensionality reduction**, which we'll see soon.

In [12]:
# find a pair of features that seem to separate the three flower species when p
# Choose any two features so that the data can be represented as a 2D plot


### Dimensionality Reduction: PCA

Principle Component Analysis (PCA) is a dimension reduction technique that can find the combinations of variables that explain the most variance.

Consider the iris dataset. It cannot be visualized in a single 2D plot, as it has 4 features. We are going to extract 2 combinations of sepal and petal dimensions to visualize it:

In [16]:
X, y = iris.data, iris.target

from sklearn.decomposition import PCA
pca = PCA(n_components=0.95)
#pca = PCA(n_components=None)
#pca = PCA(n_components=2)
pca.fit(X)
X_reduced = pca.transform(X)
print("Reduced dataset shape:", X_reduced.shape)

Reduced dataset shape: (150, 2)


In [17]:
sklearn.decomposition.PCA?

In [ ]:
plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=y,
           cmap='RdYlBu')

print("Meaning of the 2 components:")
for component in pca.components_:
    print(" + ".join("%.3f x %s" % (value, name)
                     for value, name in zip(component,
                                            iris.feature_names)))

## Supervised Learning: Classification and Regression

In **Supervised Learning**, we have a dataset consisting of both **features** and **labels**.
The task is to construct an estimator which is able to predict the label of an object
given the set of features. A relatively simple example is predicting the species of 
iris given a set of measurements of its flower. This is a relatively simple task. 
Some more complicated examples are:

- given a multicolor image of an object through a telescope, determine
  whether that object is a star, a quasar, or a galaxy.
- given a photograph of a person, identify the person in the photo.
- given a list of movies a person has watched and their personal rating
  of the movie, recommend a list of movies they would like
  (So-called *recommender systems*: a famous example is the [Netflix Prize](http://en.wikipedia.org/wiki/Netflix_prize)).
 - What are some examples of useful application in Biophysics/Bioscience research? 
 (We'll revisit this question at the end of the lecture.)
  
  Supervised learning is further broken down into two categories, **classification** and **regression**.
In classification, the label is discrete, while in regression, the label is continuous. For example,
in astronomy, the task of determining whether an object is a star, a galaxy, or a quasar is a
classification problem: the label is from three distinct categories. On the other hand, we might
wish to estimate the age of an object based on such observations: this would be a regression problem,
because the label (age) is a continuous quantity.
  
See [SciKit tutorial](https://github.com/jakevdp/sklearn_tutorial) for examples of regression. Essentially, data-fitting that we do in a typical intro physics lab class.